In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Análise de rentabilidade

Este documento replica a planilha utilizada para realizar a avaliação de rentabilidade de um paciente. O objetivo é que esta seja a base para podermos automatizar o processo. 

## Informações

Na planilha, a primeira página que merece atenção é a de "Informações". Aqui é calculado um custo fixo do paciente com base em informações conhecidas anteriormente. Abaixo temos um dicionário com todos os serviços presentes e seus respectivos valores base. Nessa página, ainda se é calculado o custo fixo total do paciente, também presente na célula abaixo. 

In [2]:
# custo fixo de cada serviço
tipos_serviço = {
    "Internação Domiciliar 6hrs": 50,
    "Internação Domiciliar 12hrs": 100,
    "Internação Domiciliar 24hrs": 200,
    "Assistência Domiciliar": 35,
    "Terapias": 9,
    "Curabem": 28.8,
    "Oxigenoterapia": 11.6,
    "Fiqbem": 3.10
}

# cálculo do custo fixo total do paciente
def custo_fixo_total(tipo_serviço: str, periodo: int):
    servico_custo = tipos_serviço[tipo_serviço]
    custo_total = servico_custo * periodo
    return custo_total

## Dados

A seguir, os dados do orçamento são inseridos na planilha a partir de um Excel ou CSV. Aqui, temos um paciente de exemplo que vem de um CSV fornecido por Thiago. 

Aqui, carregamos os dados em formato de arquivo Python, de uma forma em que cada objeto representa uma linha e cada atributo desse objeto representa uma coluna. 

In [3]:
import dados_planilha

dados_planilha.orcamento[0]

{'descricao': 'HOME CARE DIARIA COM ALTA COMPLEXIDADE',
 'quantidade': 18.0,
 'unidade': 'UN',
 'valor_unitario': '785,0000',
 'valor_total': '14.130,00',
 'grupo_procedimento': 'DIARIAS'}

## Rentabilidade

Após o carregamento dos dados, a rentabilidade é gerada. Para isso, alguns procedimentos são realizados. A principio, se separa o que será pago por pacote ou não, isto é, quais itens geram receita ou não. 

* Se um item é marcado como "sim" na coluna PACOTE, isso significa que ele está contido no pacote cobrado ao convênio. 
* Se um item é marcado como "não" na coluna PACOTE, isso indica que ele não está incluso no pacote e será cobrado ao convênio de forma a parte. Este valor entra no cálculo da receita bruta, pois representa o dinheiro que a empresa irá receber pelo item específico.

Caso o paciente não seja por pacote, imagina-se que o convênio irá pagar por cada item.

Pelos dados que vieram do CSV, não temos definidos quais itens são parte do pacote ou não. Precisamos definir como isso é determinado de maneira programática. Por enquanto, vamos considerar que se for por pacote, é por que há itens marcados com ```"grupo_procedimento" = "DIARIAS"``` nos dados. Caso contrário, entenderemos que todos os itens serão pagos pelo convênio.


In [7]:
# Verifica se o paciente é por pacote
def checar_paciente_pacote(itens_orcamento: list):
    for i in itens_orcamento:
        if i["grupo_procedimento"] == "DIARIAS":
            return True
    return False

# Trata o paciente para o caso em que ele é pacote
# TODO: ISTO PRECISA SER VALIDADO COM THIAGO
def paciente_pacote(itens_orcamento):
    for i in itens_orcamento:
        if i["grupo_procedimento"] == "DIARIAS":
            i["pacote"] = "NÃO"
        else: i["pacote"] = "SIM"
    return itens_orcamento

# Trata o paciente para o caso em que ele não é pacote
# TODO: ISTO PRECISA SER VALIDADO COM THIAGO
def paciente_nao_pacote(itens_orcamento):
    for i in itens_orcamento:
        i["pacote"] = "NÃO"
    return itens_orcamento

pacote = checar_paciente_pacote(dados_planilha.orcamento)

if pacote: itens_orcamento = paciente_pacote(dados_planilha.orcamento)
else: itens_orcamento = paciente_nao_pacote(dados_planilha.orcamento)

itens_orcamento[0]

{'descricao': 'HOME CARE DIARIA COM ALTA COMPLEXIDADE',
 'quantidade': 18.0,
 'unidade': 'UN',
 'valor_unitario': '785,0000',
 'valor_total': '14.130,00',
 'grupo_procedimento': 'DIARIAS',
 'pacote': 'NÃO'}

### Receita bruta

A receita bruta é calculada ao somar todas as fontes de receita. Isso inclui as diárias, mas também qualquer outro item marcado como ```pacote = não```. Aqui calculamos individualmente o valor de cada categoria salvamos o total.

In [ ]:
grupo_procedimentos = [
    "DIARIAS",
    "TAXAS",
    "SERV ENFERMAGEM",
    "SERVICOS DE FISIOTERAPIA",
    "SERVICOS DE FONOAUDIOLOGIA",
    "SERVICOS DE NUTRICAO",
    "SERVICOS DE TERAPIA OCUPACIONAL",
    "VISITAS E CONSULTAS MEDICAS - HC",
    "MEDICAMENTOS",
    "MEDICAMENTOS RESTRITO HOSP.",
    "ALIMENTACOES E DIETAS",
    "MATERIAIS",
    "CURATIVOS ESPECIAIS"
]

import re

def parse_valor(s):
    if s is None:
        return 0.0
    if isinstance(s, (int, float)):
        return float(s)
    s = str(s).strip()
    s = re.sub(r'[^\d,.\-()]', '', s)
    negativo = False
    if s.startswith('(') and s.endswith(')'):
        negativo = True
        s = s[1:-1]
    s = s.replace('.', '').replace(',', '.')
    try:
        val = float(s) if s not in ('', '.', '-') else 0.0
    except ValueError:
        val = 0.0
    return -val if negativo else val

def receita_bruta_categoria(itens_orcamento, categoria):
    total = 0.0
    for item in itens_orcamento:
        if item["grupo_procedimento"] == categoria and item["pacote"] == "NÃO":
            total += parse_valor(item["valor_total"])
    return total

TOTAL_RECEITA_BRUTA = 0
for categoria in grupo_procedimentos:
    rect_bruta = receita_bruta_categoria(itens_orcamento, categoria)
    TOTAL_RECEITA_BRUTA += rect_bruta
    print(f"Receita bruta {categoria}: {rect_bruta}")
print("============TOTAL============")
print(TOTAL_RECEITA_BRUTA)

Receita bruta DIARIAS: 14130.0
Receita bruta TAXAS: 0.0
Receita bruta SERV ENFERMAGEM: 0.0
Receita bruta SERVICOS DE FISIOTERAPIA: 0.0
Receita bruta SERVICOS DE FONOAUDIOLOGIA: 0.0
Receita bruta SERVICOS DE NUTRICAO: 0.0
Receita bruta SERVICOS DE TERAPIA OCUPACIONAL: 0.0
Receita bruta VISITAS E CONSULTAS MEDICAS - HC: 0.0
Receita bruta MEDICAMENTOS: 0.0
Receita bruta MEDICAMENTOS RESTRITO HOSP.: 0.0
Receita bruta ALIMENTACOES E DIETAS: 0.0
Receita bruta MATERIAIS: 0.0
Receita bruta CURATIVOS ESPECIAIS: 0.0
============TOTAL============
14130.0
